In [30]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
%matplotlib inline
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import pyfolio as pf
import yfinance as yf
from pylab import rcParams
rcParams['figure.figsize']= 16,12
rcParams['figure.facecolor']= '#eeeeee'
plt.title('dummy')
plt.plot([1,3,2,4])
plt.close()


# silence warnings
import warnings
warnings.filterwarnings('ignore')
# plt.style.use('ggplot')

In [31]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

            # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:
            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    data_folder = '/Users/shrikanthnallapati/Projects/stock_app/Data'
    data_df = pd.read_csv(os.path.join(data_folder, "DE.csv"))
    data_df.drop('Adj Close', axis=1)
    data_df['Datetime'] = pd.to_datetime(data_df['Datetime'],utc=True)
    #Set index as Datetime
    data_df= data_df.set_index('Datetime')
    # AMZN_1h_df.head()
    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')

    # Create a Data Feed
    data = bt.feeds.PandasData(dataname = data_df)
    # data = bt.feeds.PandasData(
    #     dataname=AMZN_1h_df,
    #     # Do not pass values before this date
    #     fromdate=datetime.datetime(2000, 1, 1),
    #     # Do not pass values before this date
    #     todate=datetime.datetime(2000, 12, 31),
    #     # Do not pass values after this date
    #     reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(15000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=35)

    # Set the commission
    cerebro.broker.setcommission(commission=0.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    stats = cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Plot the result
    cerebro.plot(iplot=True, volume= False, style = 'lines')


Starting Portfolio Value: 15000.00
2023-10-10, Close, 387.46
2023-10-10, BUY CREATE, 387.46
2023-10-10, BUY EXECUTED, Price: 387.46, Cost: 13561.10, Comm 0.00
2023-10-10, Close, 387.28
2023-10-10, Close, 387.53
2023-10-10, Close, 387.62
2023-10-10, Close, 387.60
2023-10-10, Close, 387.37
2023-10-10, Close, 387.38
2023-10-10, Close, 387.70
2023-10-10, Close, 387.56
2023-10-10, Close, 387.70
2023-10-10, Close, 387.85
2023-10-10, Close, 387.68
2023-10-10, Close, 387.92
2023-10-10, Close, 387.80
2023-10-10, Close, 387.89
2023-10-10, Close, 387.75
2023-10-10, Close, 388.01
2023-10-10, Close, 388.28
2023-10-10, Close, 387.92
2023-10-10, Close, 387.85
2023-10-10, Close, 387.81
2023-10-10, Close, 387.91
2023-10-10, Close, 388.10
2023-10-10, Close, 388.09
2023-10-10, Close, 388.14
2023-10-10, Close, 388.14
2023-10-10, Close, 388.20
2023-10-10, Close, 388.29
2023-10-10, Close, 388.34
2023-10-10, Close, 388.32
2023-10-10, Close, 387.96
2023-10-10, SELL CREATE, 387.96
2023-10-10, SELL EXECUTED, Pr

<IPython.core.display.Javascript object>

In [32]:
# Get Results from the strat object after backtest finished
strat = stats[0]

# Create pyfoliozer object from strategy results object
pyfoliozer = strat.analyzers.getbyname('pyfolio')

# Get returns, positions, transactions and gross_lev
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

# get_pf_items()

# Visualize 
pf.create_full_tear_sheet(returns,positions=positions,transactions=transactions)
   


ValueError: 'pyfolio' is not in list